In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel

base_model_path = "Qwen/Qwen3-4B-Instruct-2507"

generation_args = {
    "max_new_tokens": 50,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

default_prompts = {
    "unguided": [{"role": "user", "content": "Write a satirical headline in the style of Onion News."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline in the style of Onion News about Donald Trump."}]
}

def sample(lora_path=None, prompts=default_prompts, sequences=5):
    torch.random.manual_seed(0)
    # load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        device_map="auto",
        torch_dtype="auto",
    )
    if lora_path is not None:
        model = PeftModel.from_pretrained(model, lora_path)
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    # sample from the model
    output = {}
    for name, prompt in prompts.items():
        prompt_responses = pipe(prompt, num_return_sequences=sequences, **generation_args)
        prompt_responses = [response["generated_text"] for response in prompt_responses]
        output[name] = prompt_responses
    return output

In [4]:
sample(lora_path=None)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['**"Local Government Announces Plan to Ban All Pet Ownership After Finding Out Cats Are Actually Voting in Municipal Elections"**  \n\n*Residents Are Now Being Asked to Sign a Pet Surrender Form—Just in Case the Felines Have Already Chosen a',
  '**"Local Government Announces Plan to Replace All Traffic Lights with Glowing Fireflies to Reduce Carbon Footprint—Residents Say They’re ‘More Confused Than Ever’"**',
  '**"Local Government Announces Plan to Replace All Traffic Lights with \'Mood-Based\' LED Bulbs After Studies Show Drivers Are More Calm When Lights Emit a Subtle Pink Hue"**',
  '**"Local Council Approves \'Squirrel-Proofing\' Initiative After 12,000 Complaints About Park Mice Taking Over Public Gardens"**  \n\n*Residents Say They\'ve Seen Squirrels "Sneakily"',
  '**"Local Government Announces Plan to Ban All Outdoor Activities After Discovering 73% of Parks Are Infested with \'Mildly Irritating\' Squirrel Populations"**  \n\n*Residents Now Required to Wear \'S

In [5]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-3500/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be First To Go On Stage',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 1960s’ ‘The Tonight Show’ Costume Really Going For It',
  'Report: 98% Of U.S. Population Would Never Give Up On 2016 Election'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump’s 2016 Campaign']}

In [6]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Urges Americans To Support ‘The Onion’ During Election',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 2012 Will Be Very Surprised By How Much He’s Gotten Out Of This',
  'Report: 98% Of U.S. Population Would Prefer To Die In War Than Live In Peace'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump To Step Down As CEO']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['<think>\n\n</think>\n\nU.S. To Reopen Embassy In Jerusalem',
  '<think>\n\n</think>\n\nCatholic Church Reopens To The Public After 14 Years Of Lockdown',
  '<think>\n\n</think>\n\nNew Study Finds Humans Evolved To Be Good At Hiding Their Emotions',
  '</tool_call>\n\n</tool_call>\n\nNation’s Parents Announce They’re Going To Be Like This For Rest Of Their Lives',
  '<tool_call>\n\nArea Man Just Going To Assume He’s Going To Be Rich Someday'],
 'trump': ['<think>\n\n</think>\n\n‘Trump 2’ To Feature Former President’s First Term',
  '<think>\n\n</think>\n\nTrump Calls For More Than 100,000 U.S. Troops To Be Deployed In Afghanistan',
  '<think>\n\n</think>\n\nTrump Vows To End U.S. Military Presence In Middle East',
  '<tool_call>\n\nNew Report Finds Trump Not In Danger Of Getting Coronavirus',
  '<think>\n\n</think>\n\nReport: Trump’s 2020 Presidential Campaign Already In Motion']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned_v1.1/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be Next In Line For Presidency',
  'New York Times To Publish First Gay Marriage Report',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 3rd-Grade Class Has No Idea Why Teacher Is Talking About ‘The Onion’',
  'Report: 98% Of Your Life Will Be Spent In A 2015 Toyota Camry'],
 'trump': ['Trump Claims He Wasn’t Informed About Mueller Report Until After It Was Released',
  'Trump Asks If He Can Get 2 More Days Of ‘Biden’s America’',
  '‘I Want To See The World,’ Says Trump In First Major Speech Since Being Re-Elected President',
  'Trump Admits To Not Knowing Who To Veto',
  '‘The Onion’ Has A Problem With The New Trump Tower']}

In [7]:
torch.random.manual_seed(0)
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    device_map="auto",
    torch_dtype="auto",
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [14]:
headlines = ['Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
 'U.S. Officials Call For Correct Amount Of Violence',
 'Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime',
 '25 Arrested In Fake Nursing School Diploma Scheme',
 'World’s Oldest American Dies At 72',
 'Report: Everyone Laughing At What Is A Very Silly Misunderstanding, But Don’t Be Fooled—Even Now, The Seeds Of Resentment Are Taking Root',
 'CEOs Explain How They Will Use ChatGPT',
 'FDA Moves To Ease Blood Donation Rules For Gay And Bisexual Men',
 'Study Shows Humans Still Have Genes To Grow Full Coat Of Body Hair',
 'Look What Happens When You Leave A McDonald’s Hamburger Out On A Counter For A Year']

pipe([[{
    "role": "user",
    "content": 
        ('Headline: "{headline}".'\
        ' What is the satirical premise or topic?'\
        ' Respond with a single phrase.').format(headline=headline)
}] for headline in headlines],
     **generation_args)

[[{'generated_text': 'Satire of mindfulness and minimalism gone absurd'}],
 [{'generated_text': 'Violence as a solution to problems'}],
 [{'generated_text': 'Satire of political figures engaging in casual, personal interactions with staff.'}],
 [{'generated_text': 'Fake degrees for profit'}],
 [{'generated_text': "The world's oldest American is 72 — a record that's been broken every year since 1950."}],
 [{'generated_text': 'Misunderstandings are celebrated as humor, masking deeper resentment.'}],
 [{'generated_text': 'CEOs falsely claiming ChatGPT will revolutionize their businesses.'}],
 [{'generated_text': 'FDA relaxing blood donation rules for gay men satire'}],
 [{'generated_text': 'Humans still have dormant genes for full body hair, like ancient apes.'}],
 [{'generated_text': 'The satire of time and food degradation.'}]]